In [53]:
# need to use the functions in single_layer_interval_propagation
%run single_layer_interval_propagation.ipynb

In [54]:
# description: compute the output interval for simple interval propagation
# the input parameters layers: contains layer, layer is also a list datatype. in layer there are w and b. the datatype of layers is list, layer is list, layers[i][0] = w is ndarray, layers[i][1] is ndarray
# the input parameters activation_functions: str list, contains the activation functions defined in each layer.
# the input parameters input_min and input_max: minimum and maximum value vectors to define the input interval
# return output_min and output_max: the output interval of nn_model
def simple_interval_propagation(layers, activation_functions, input_min, input_max):
    min_vec_run = input_min
    max_vec_run = input_max
    i = 0
    for layer in layers:
        min_vec_run, max_vec_run = get_min_max_for_single_layer(layer[0], layer[1], activation_functions[i], min_vec_run, max_vec_run)
        i = i+1
    output_min = min_vec_run
    output_max = max_vec_run
    return output_min, output_max

In [ ]:
# description: compute the output interval for simple interval propagation
# input parameter nn_model: a keras model
# input parameter input_min and input_max: minimum and maximum value vectors to define the input interval/region
# return output_min and output_max: the output interval/region of nn_model
def simple_interval_propagation_model(nn_model, input_min, input_max):
    # get layers(ws and bs) from nn_model 
    layers=[]
    for layer in nn_model.layers:
        layers.append(layer.get_weights())
        
    # get activation_functions from nn_model
    activation_functions=[]
    for layer in nn_model.layers:
        activation_functions.append(layer.get_config()["activation"])
    
    output_min, output_max = simple_interval_propagation(layers, activation_functions, input_min, input_max)
    
    return output_min, output_max

In [55]:
# description: split the input interval/region into two evenly spaced intervals/regions
# input parameters min_vec and max_vec: minimum and maximum value vectors to define the input interval/region
# return interval1 and interval2: two evenly spaced intervals
def split_2(min_vec, max_vec):
    # check every elements in min_vec is smaller than max_vec
    assert(np.all(min_vec<=max_vec))
    
    num_dim = min_vec.shape[1]
    min_vec = min_vec.reshape((1, num_dim))
    max_vec = max_vec.reshape((1, num_dim))
    # the index of the largest gap
    dif_vec = max_vec - min_vec
    dif_max_index = dif_vec.argmax()
    # get the mid value of the larfest gap
    event_space = np.linspace(min_vec[0][dif_max_index], max_vec[0][dif_max_index], 3)
    mid = event_space[1]  
    # get the new max_vec1 for region1 and min_vec2 for region2
    # region1
    min_vec1 = min_vec.copy()
    max_vec1 = max_vec.copy()
    max_vec1[0][dif_max_index] = mid
    #region2
    min_vec2 = min_vec.copy()
    max_vec2 = max_vec.copy()
    min_vec2[0][dif_max_index] = mid
    
    interval1 = [min_vec1, max_vec1]
    interval2 = [min_vec2, max_vec2]
    return interval1, interval2

In [5]:
# description: find the center point of a interval/region
# input parameter region: it is a region, its type is a list contains two np.ndarray, one array is min, another one is max
# return center_point: is a numpy array represents the center point of the input region
def find_center_point(region):
    min_vec = region[0]
    max_vec = region[1]
    num_dims = min_vec.shape[1]
    assert(min_vec.shape == (1, num_dims))
    assert(max_vec.shape == (1, num_dims))
    assert(np.all(min_vec <= max_vec))
    
    center_point = np.zeros((1,num_dims))
    for dim in range(num_dims):
        center_point[0][dim] = np.linspace(min_vec[0][dim], max_vec[0][dim], 3)[1]
    assert(center_point.shape == min_vec.shape)
    return center_point

In [1]:
# description: find all the counter examples in the waiting list 
# input parameter nn model: a keras model
# input parameter all_regions: a list that contains all the regions that are still in the waiting list
# return counter_example_list: a list that contains all the counter examples(counter points)
def find_counter_examples_in_all_regions_in_list_model_1(nn_model, all_regions):
    counter_example_list = []
    for region in all_regions:
        region_center_point = find_center_point(region)
        predict_prob = nn_model.predict(region_center_point)
        if predict_prob.argmax() != 5:
            counter_example_list.append(region_center_point)
    return counter_example_list

In [1]:
# description: increase the brightness(values) of non zero pixels
# input parameters min_vec: the original image in shape of (n, 1)
# input parameters add_num: the value of brightness to be increased
# return max_vec: the image that have had its brightness increased, this vector will be used as the maximum value of input interval
def mnist_get_max(min_vec, add_num):
    assert(add_num >= 0)
    max_vec = min_vec.copy()
    temp_num = 255 - add_num
    num_pixels = min_vec.shape[1]
    
    for row in range(min_vec.shape[1]):
        if min_vec[0][row] == 0:
            max_vec[0][row] = 0
        elif min_vec[0][row] >= temp_num:
            max_vec[0][row] = 255
        else:
            max_vec[0][row] = min_vec[0][row] + add_num
    return max_vec

In [2]:
# description: check specification of model 1 for the output interval. For lable 5, its minimum prob value should larger than the other maximum prob values
# input parameter output_min, output_max: minimum and maximum value vectors to define the output interval, after the simple interval propagation
# return true if 
def check_specification_model_1(output_min, output_max):
    assert(np.all(output_min <= output_max))
    min_vec = output_min.copy()
    max_vec = output_max.copy()
    num_dim = output_min.shape[1]
    min_vec.reshape((1, num_dim))
    max_vec.reshape((1, num_dim))
    
    # set output_max[6]=0, than all the values in output_max should smaller than output_min[6]
    max_vec[0][5] = 0
    return np.all(min_vec[0][5] > max_vec)

In [ ]:
# description: check is any counter example inside the TODOlist. find one counter example directly return.
# input parameter nn_model the keras model
# input parameter all_regions is a list, in the list, the smaller values should on the top/back of the TODO list
# return a point
def if_any_counter_example_model_1_small_first(nn_model, all_regions):
    counter_example = []
    for region in reversed(all_regions):
        region_center_point = find_center_point(region)
        predict_prob = nn_model.predict(region_center_point)
        if predict_prob.argmax() != 5:
            counter_example.append(region_center_point)
            break
    return counter_example

In [ ]:
# description: check is any counter example inside the TODOlist. find one counter example directly return.
# input parameter nn_model the keras model
# input parameter all_regions is a list, in the list, the smaller values should on the top/back of the TODO list
# return a point
def if_any_counter_example_model_1_large_first(nn_model, all_regions):
    counter_example = []
    for region in all_regions:
        region_center_point = find_center_point(region)
        predict_prob = nn_model.predict(region_center_point)
        if predict_prob.argmax() != 5:
            counter_example.append(region_center_point)
            break
    return counter_example

In [ ]:
# description: get a list of intervals that generate by using l infinity norm. the maximum perturbation of l_infinity_interval_list is smaller than
# the input parameters point_list: a list contains all the points
# the input parameters max_perturbation: the maximum perturbation of l infinity norm, to generate the interval
# return output_l_infinity_interval_list: a list of all intervals that are performed l infinity norm
def get_l_infinity_list(orig_list, max_perturbation):
    assert(max_perturbation >= 0)
    l_infinity_intervals = []
    for element in orig_list:
        interval_min = element - max_perturbation
        interval_max = element + max_perturbation
        input_interval = [interval_min, interval_max]
        l_infinity_intervals.append(input_interval)
    assert(len(orig_list) == len(l_infinity_intervals))
    return l_infinity_intervals